<a href="https://colab.research.google.com/github/camila-cg/analise-dados-educacionais/blob/main/Teste_an%C3%A1lise_base_educacional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bibliotecas utilizadas

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


Fixando semente para reprodutibilidade dos resultados

In [3]:
np.random.seed(42)

Leitura dos dados

In [4]:
dataset_mat = pd.read_csv("https://raw.githubusercontent.com/camila-cg/analise-dados-educacionais/main/student-mat.csv", sep = ';')
dataset_port = pd.read_csv("https://raw.githubusercontent.com/camila-cg/analise-dados-educacionais/main/student-por.csv", sep = ';')

dataset_mat.head(2)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6


## Pré-Processamento

Verificando a presença de dados faltantes na base de dados

In [10]:
print(dataset_mat.info())
print(dataset_port.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 34 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      395 non-null    object
 1   sex         395 non-null    object
 2   age         395 non-null    int64 
 3   address     395 non-null    object
 4   famsize     395 non-null    object
 5   Pstatus     395 non-null    object
 6   Medu        395 non-null    int64 
 7   Fedu        395 non-null    int64 
 8   Mjob        395 non-null    object
 9   Fjob        395 non-null    object
 10  reason      395 non-null    object
 11  guardian    395 non-null    object
 12  traveltime  395 non-null    int64 
 13  studytime   395 non-null    int64 
 14  failures    395 non-null    int64 
 15  schoolsup   395 non-null    object
 16  famsup      395 non-null    object
 17  paid        395 non-null    object
 18  activities  395 non-null    object
 19  nursery     395 non-null    object
 20  higher    

Verificamos que as bases a serem utilizadas não possuem valores faltantes.

### Transformação dos Dados

Os atributos que assumem apenas dois valores possíveis, como os valores **'yes'** e **'no'** serão transformados em valores binários, dado que os modelos de aprendizado de máquina não conseguem trabalhar com strings, apenas números

In [31]:
def labelEncoder(df, colunas, filename):
  label_df = df.copy()
  le = LabelEncoder()
  
  for col in colunas:
    label_df[col] = le.fit_transform(label_df[col])
    le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    # print(col)
    # print(le_name_mapping)
    arq = open(filename, 'a')
    print("Coluna: ", col, file=arq)
    print(le_name_mapping, file=arq)
    print('\n', file=arq)
    arq.close()
  return label_df

colunas_bin = ["school", "sex", "schoolsup", "famsup", "paid", "activities", "nursery", "higher", "internet", "romantic", "address", "famsize", "Pstatus"]

dataset_mat_proc = labelEncoder(dataset_mat, colunas_bin, 'label_encoder.txt')
dataset_port_proc = labelEncoder(dataset_port, colunas_bin, 'label_encoder.txt')

print("Primeira linha do dataset de matemática após a conversão dos atributos binários:")
dataset_mat_proc.head(1)

Primeira linha do dataset de matemática após a conversão dos atributos binários:


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,aprovado
0,0,0,18,1,0,0,4,4,at_home,teacher,course,mother,2,2,0,1,0,0,0,1,1,0,0,4,3,4,1,1,3,6,5,6,6,0


Agora vamos converter os atributos categóricos que assumem mais do que dois valores possíveis em valores discretos.

In [33]:
colunas_cat = ["Mjob", "Fjob", "reason", "guardian"]

dataset_mat_proc = labelEncoder(dataset_mat_proc, colunas_cat, 'label_cat.txt')
dataset_port_proc = labelEncoder(dataset_port_proc, colunas_cat, 'label_cat.txt')

print("Primeira linha do dataset de matemática após a conversão dos atributos categóricos:")
dataset_mat_proc.head(1)

Primeira linha do dataset de matemática após a conversão dos atributos categóricos:


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,aprovado
0,0,0,18,1,0,0,4,4,0,4,0,1,2,2,0,1,0,0,0,1,1,0,0,4,3,4,1,1,3,6,5,6,6,0


In [34]:
dataset_mat_proc['G1'].value_counts()

10    51
8     41
11    39
7     37
12    35
13    33
9     31
14    30
15    24
6     24
16    22
18     8
17     8
5      7
19     3
4      1
3      1
Name: G1, dtype: int64

Será criada uma **classe binária** que indicará se o aluno foi aprovado na disciplina ou não, onde os modelos de classificação deverão prever com base nas informações disponíveis sobre cada aluno se ele será ou não aprovado.

In [35]:
dataset_mat_proc['aprovado'] = ''
dataset_mat_proc.loc[(dataset_mat_proc.G3 >= 10), 'aprovado'] = 1
dataset_mat_proc.loc[(dataset_mat_proc.G3 < 10), 'aprovado'] = 0

dataset_port_proc['aprovado'] = ''
dataset_port_proc.loc[(dataset_port_proc.G3 >= 10), 'aprovado'] = 1
dataset_port_proc.loc[(dataset_port_proc.G3 < 10), 'aprovado'] = 0
dataset_port_proc.head(2)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,aprovado
0,0,0,18,1,0,0,4,4,0,4,0,1,2,2,0,1,0,0,0,1,1,0,0,4,3,4,1,1,3,4,0,11,11,1
1,0,0,17,1,0,1,1,1,0,2,0,0,1,2,0,0,1,0,0,0,1,1,0,5,3,3,1,1,3,2,9,11,11,1


Será criada também uma coluna que indicará o conceito final do aluno, seguindo a seguinte regra:
- Notas entre [16, 20]: conceito **A**,
- Notas entre [14, 16): conceito **B**,
- Notas entre [12, 14): conceito **C**,
- Notas entre [10, 12): conceito **D** e
- Notas entre [0, 10): conceito **F**

In [39]:
dataset_mat_proc['conceito'] = ''
dataset_mat_proc.loc[(dataset_mat_proc.G3 >= 0) & (dataset_mat_proc.G3 < 10), 'conceito'] = 'F'
dataset_mat_proc.loc[(dataset_mat_proc.G3 >= 10) & (dataset_mat_proc.G3 < 12), 'conceito'] = 'D'
dataset_mat_proc.loc[(dataset_mat_proc.G3 >= 12) & (dataset_mat_proc.G3 < 14), 'conceito'] = 'C'
dataset_mat_proc.loc[(dataset_mat_proc.G3 >= 14) & (dataset_mat_proc.G3 < 16), 'conceito'] = 'B'
dataset_mat_proc.loc[(dataset_mat_proc.G3 >= 16) & (dataset_mat_proc.G3 <= 20), 'conceito'] = 'A'

dataset_port_proc['conceito'] = ''
dataset_port_proc.loc[(dataset_port_proc.G3 >= 0) & (dataset_port_proc.G3 < 10), 'conceito'] = 'F'
dataset_port_proc.loc[(dataset_port_proc.G3 >= 10) & (dataset_port_proc.G3 < 12), 'conceito'] = 'D'
dataset_port_proc.loc[(dataset_port_proc.G3 >= 12) & (dataset_port_proc.G3 < 14), 'conceito'] = 'C'
dataset_port_proc.loc[(dataset_port_proc.G3 >= 14) & (dataset_port_proc.G3 < 16), 'conceito'] = 'B'
dataset_port_proc.loc[(dataset_port_proc.G3 >= 16) & (dataset_port_proc.G3 <= 20), 'conceito'] = 'A'
dataset_port_proc.head(3)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,aprovado,conceito
0,0,0,18,1,0,0,4,4,0,4,0,1,2,2,0,1,0,0,0,1,1,0,0,4,3,4,1,1,3,4,0,11,11,1,D
1,0,0,17,1,0,1,1,1,0,2,0,0,1,2,0,0,1,0,0,0,1,1,0,5,3,3,1,1,3,2,9,11,11,1,D
2,0,0,15,1,1,1,1,1,0,2,2,1,1,2,0,1,0,0,0,1,1,1,0,4,3,2,2,3,3,6,12,13,12,1,C


##Divisão da base

Divisão da base em treino, validação e teste (60, 20, 20)


In [7]:
treino_mat, validacao_mat, teste_mat = np.split(dataset_mat.sample(frac=1), [int(.6*len(dataset_mat)), int(.8*len(dataset_mat))])
treino_port, validacao_port, teste_port = np.split(dataset_port.sample(frac=1), [int(.6*len(dataset_port)), int(.8*len(dataset_port))])


# Classificação da situação final do aluno

##Modelos de classificação

In [8]:
from sklearn import tree
model = tree.DecisionTreeClassifier()


#Previsão da nota final do aluno